# Imports
All problem-speciffic files (code) in in the folder `GinzburgLandau2D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [ ]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from GinzburgLandau2D.jax_energy import J
from GinzburgLandau2D.mesh import MeshGL2D

from jax import config

config.update("jax_enable_x64", True)

# Setup of the problem

In [ ]:
mesh = MeshGL2D(mesh_level=7)
params, adjacency, u_init = mesh.get_data_jax()
energy_GL = EnergyDerivator(J, params, adjacency, u_init)

F, dF, ddF = energy_GL.get_derivatives()
ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", verbose=True, tol=1e-3)
print(energy_GL.timings)

# Solution via Newton's method

In [ ]:
res = newton(F, dF, ddf_with_solver, u_init, verbose=True, tolf=1e-6, tolg=1e-5, linesearch_tol=1e-3)
res